**1 数据导入与清洗**

In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [132]:
path='./tmall_order_report.csv'
data=pd.read_csv(path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      28010 non-null  int64  
 1   总金额       28010 non-null  float64
 2   买家实际支付金额  28010 non-null  float64
 3   收货地址      28010 non-null  object 
 4   订单创建时间    28010 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


In [133]:
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2020-02-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8


付款时间存在NA值：未付款订单

In [134]:
#列名空格清除
data.columns=data.columns.str.strip()
data.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址', '订单创建时间', '订单付款时间', '退款金额'], dtype='object')

In [135]:
#规范收货地址名称，以便可视化处理
add_city=['北京','上海','天津','重庆']
for a in add_city:
    data['收货地址']=data['收货地址'].str.replace(a,a+'市')
data['收货地址'].unique()

array(['上海市', '内蒙古自治区', '安徽省', '湖南省', '江苏省', '浙江省', '天津市', '北京市', '四川省',
       '贵州省', '辽宁省', '河南省', '广西壮族自治区', '广东省', '福建省', '海南省', '江西省', '甘肃省',
       '河北省', '黑龙江省', '云南省', '重庆市', '山西省', '吉林省', '山东省', '陕西省', '湖北省',
       '青海省', '新疆维吾尔自治区', '宁夏回族自治区', '西藏自治区'], dtype=object)

**2 数据可视化**

In [136]:
result={}
result['总订单数']=data['订单编号'].count()
result['已完成订单数']=data['订单付款时间'][data['订单付款时间'].notnull()].count()
result['未付款订单数']=data['订单编号'][data['订单付款时间'].isnull()].count()
result['退款订单数']=data['退款金额'][data['退款金额']>0].count()
result['总退款金额']=data['退款金额'][data['退款金额']>0].sum()
result['总订单金额']=data['总金额'][data['订单付款时间'].notnull()].sum()
result['总收入金额']=data['买家实际支付金额'].sum()
result

{'总订单数': 28010,
 '已完成订单数': 24087,
 '未付款订单数': 3923,
 '退款订单数': 5646,
 '总退款金额': 572335.92,
 '总订单金额': 2474823.0700000003,
 '总收入金额': 1902487.15}

In [137]:
from pyecharts import options as opts
from pyecharts.charts import Map, Bar, Line
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts,VisualMapOpts
from pyecharts.faker import Faker

**2.1 整体情况总览**

In [138]:
table=Table()
headers=['总订单数', '总订单金额', '已完成订单数', '总实际收入金额', '退款订单数', '总退款金额', '成交率', '退货率']
rows=[
    [
        result['总订单数'],
        f"{result['总订单金额']/10000:.2f}万",
        result['已完成订单数'],
        f"{result['总收入金额']/10000:.2f}万",
        result['退款订单数'],
        f"{result['总退款金额']/10000:.2f}万",
        f"{result['已完成订单数']/result['总订单数']:.2%}",
        f"{result['退款订单数']/result['已完成订单数']:.2%}"
    ]
]
table.add(headers,rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title='整体情况总览')
)
table.render_notebook()

总订单数,总订单金额,已完成订单数,总实际收入金额,退款订单数,总退款金额,成交率,退货率
28010,247.48万,24087,190.25万,5646,57.23万,85.99%,23.44%


整体情况的每个单项均可与该商品的月历史数据波动或设定目标进行比较，从而找出可优化点，做出相应调整。
如假设：对比历史数据/目标数据，本月的退货率上升（偏高）/成交率下降（偏低）/总订单数减少等
优化思路：
1.明确对象：对商品分组聚合进行深入分析，确定出现该消极变化的具体类目；
2.分向求因
2.1产品：对对应类目的商品进行价格、质量、流量、舆情等因素的波动监测
2.2顾客：将顾客分类为新客/回头客，分别提取对应数据
2.3店铺：对店铺的推广方式、促销活动、运营热度等因素进行分析
3.筛选真因：多维度分别选取相关性最强的一个或数个因素，提出针对性的改善对策，协调其他部门共同改善
同理，若发生退货率下降/成交率上升/总订单数增加等积极变化，亦可对上述指标进行深入探索，以期将该积极变化映射到其他商品/店铺上。

**2.2 地区分析**

In [145]:
result_area=data[data['订单付款时间'].notnull()].groupby('收货地址').agg({'订单编号':'count'}).to_dict()['订单编号']
area=(
    Map().add('订单量',[*result_area.items()],'china',is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='地区分布'),
        visualmap_opts=opts.VisualMapOpts(max_=2000)
    )
)
area.render('area_analysis.html')

'c:\\Users\\86155\\Desktop\\Tmall\\area_analysis.html'

从地图上可以看出，上海、广东、北京、江苏、浙江、四川省市的订单量位列前茅，而西北地区的订单量则明显处于较低的水平。
其原因可能与发货地、物流便利程度、运费等因素有关，可考虑增加仓库点位、与物流商洽谈合作等。

**2.3 时间分析**

In [140]:
data['订单付款时间']=pd.to_datetime(data['订单付款时间'])
data['订单创建时间']=pd.to_datetime(data['订单创建时间'])

In [141]:
#月订单量走势分析
result_date=data.groupby(data['订单创建时间'].apply(lambda x:x.strftime('%Y-%m-%d'))).agg({'订单编号':'count'}).to_dict()['订单编号']
date=(
    Line()
    .add_xaxis(list(result_date.keys()))
    .add_yaxis('订单量',list(result_date.values()))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(
                    type_='max',name='最大值'
                )
            ]
        )
    )
    .set_global_opts(title_opts=opts.TitleOpts(title='订单量日走势'))
)
date.render('date_analysis.html')

'c:\\Users\\86155\\Desktop\\Tmall\\date_analysis.html'

在2月每日的订单量数据波动中，可以看到17号之前都处于较低的水平，从17号始有了明显的增长趋势，在25号到达了峰值。
其原因可能与疫情封控、物流受限有关。

In [142]:
#日订单量走势分析
result_time=data.groupby(data['订单创建时间'].apply(lambda x:x.strftime('%H'))).agg({'订单编号':'count'}).to_dict()['订单编号']
x=[*result_time.keys()]
y=[*result_time.values()]
time=(
    Bar()
    .add_xaxis(x)
    .add_yaxis('订单量',y)
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(name='峰值',type_='max')
            ]
        )
    )
)
time.render('time_analysis.html')

'c:\\Users\\86155\\Desktop\\Tmall\\time_analysis.html'

从每小时的订单量走势来看，一天有3个下单高峰期，分别是10、15、21点，在凌晨5点左右下单量处于低谷期。
可建议客服部门在下单高峰期提高运力，保证回复顾客的及时性，以提高下单量。

In [143]:
#付款时间分析
d=data['订单付款时间']-data['订单创建时间']
d[d.notnull()].apply(lambda x:x.seconds/60).mean()

7.7399046511949745

顾客的平均付款时间为7.7分钟，其波动可能与商品价格、促销活动、商品描述、商品评价等指标有关。
可通过优化商品详情进行改善。